In [19]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable, SubplotDivider, Size
from mpl_toolkits.axes_grid1.mpl_axes import Axes
import matplotlib.ticker as mticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib as mpl
from matplotlib.offsetbox import AnchoredText

aea_crs = ccrs.AlbersEqualArea(central_longitude=-96,
                               standard_parallels=(29.5, 70),
                               central_latitude=37.5,
                               false_easting=0,
                               false_northing=0)
# to proj4 form
aea_crs_proj4 = aea_crs.proj4_init

%matplotlib inline

import glob
from tqdm.notebook import tqdm

import geopandas as gpd
import numpy as np
from numpy import (arctan2, cos, sin, sqrt,
                    pi, append, diff)
from shapely.geometry import MultiPolygon
import pandas as pd

from src.geo_processing import select_big_from_MP
from src.geo_plot import create_gdf_for_plot

plt.style.use('ggplot')

### Read basic geometry

In [20]:
geometry_data = gpd.read_file(
    './geo_data/research_ws/HCDN_nhru_final_671.shp')
geometry_data['geometry'] = [select_big_from_MP(ws)
                             for ws in geometry_data['geometry']]
geometry_data['centroids'] = [geom.centroid for geom 
                              in geometry_data['geometry']]

geometry_data['hru_id'] = ['0' + str(gauge) if len(str(gauge)) != 8
                           else str(gauge)
                           for gauge in geometry_data['hru_id']]
geometry_data = geometry_data[['hru_id', 'centroids']]
geometry_data = geometry_data.rename(columns={'hru_id': 'gauge_id'})
geometry_data = geometry_data.set_index('gauge_id')
# select basemap
basemap_data = gpd.read_file(
    './geo_data//basemap/cb_2018_us_state_500k.shp')
# area of watersheds
area_data = pd.read_csv('./attributes/static_data.csv')

/home/dima/miniconda3/envs/geo/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [29]:
area_data = pd.read_csv('./attributes/static_data.csv')
area_data['gauge_id'] = ['0' + str(gauge) if len(str(gauge)) != 8
                           else str(gauge)
                           for gauge in area_data['gauge_id']]
area_data = area_data.set_index('gauge_id')

### Read model results

In [18]:
test_ERA = pd.read_csv(
    './runs/ERA_HydroATLAS/test/model_epoch030/test_metrics.csv')
test_ERA['basin'] = ['0' + str(gauge) if len(str(gauge)) != 8
                     else str(gauge)
                     for gauge in test_ERA['basin']]
test_ERA = test_ERA.rename(columns={'basin': 'gauge_id'})
test_ERA = test_ERA.set_index('gauge_id')

test_Maurer = pd.read_csv(
    './runs/Maurer_CAMELS/test/model_epoch030/test_metrics.csv')
test_Maurer['basin'] = ['0' + str(gauge) if len(str(gauge)) != 8
                        else str(gauge)
                        for gauge in test_Maurer['basin']]
test_Maurer = test_Maurer.rename(columns={'basin': 'gauge_id'})
test_Maurer = test_Maurer.set_index('gauge_id')


### Prepare for plotting ..

In [31]:
Maurer_report = create_gdf_for_plot(
    df=geometry_data.join(test_Maurer).dropna(),
    geom_df=geometry_data).set_crs(epsg=4326).to_crs(aea_crs_proj4)
Maurer_report = Maurer_report.join(area_data[['ws_area']])

Era_report = create_gdf_for_plot(
    df=geometry_data.join(test_ERA).dropna(),
    geom_df=geometry_data).set_crs(epsg=4326).to_crs(aea_crs_proj4)
Era_report = Era_report.join(area_data[['ws_area']])
Era_report

,centroids,NSE,KGE,geometry,ws_area
gauge_id,,,,,
01022500,POINT (-68.07312697717823 44.79691442499065),0.771757,0.829019,POINT (2055384.540 1208048.100),620.423845
01031500,POINT (-69.58118579873592 45.235684060414336),0.648758,0.814191,POINT (1931914.466 1218908.881),766.509220
01047000,POINT (-70.16212994563486 44.9874360973794),0.657593,0.810694,POINT (1900170.183 1177448.102),904.969099
01052500,POINT (-71.17196580180823 44.961677816296955),0.690921,0.782483,POINT (1829278.381 1151036.994),396.118340
01054200,POINT (-71.05872447788511 44.31072366684487),0.543265,0.528152,POINT (1860720.364 1081136.294),181.360027
...,...,...,...,...,...
14309500,POINT (-123.74216186465165 42.83016828245654),0.812787,0.773843,POINT (-2120414.383 987335.242),226.431942
14316700,POINT (-122.64677479237939 43.45289842207722),0.804729,0.745503,POINT (-2016139.361 1027444.465),588.244890
14325000,POINT (-124.02763483539653 42.79248568397815),0.824412,0.770569,POINT (-2142844.852 990819.059),445.160763
